<a href="https://colab.research.google.com/github/simon-ru/An-Operational-Analysis-of-Lake-Surface-Water-Temperature/blob/main/ML_Crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta
from math import isinf

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Binance_BTCUSDT_d.csv to Binance_BTCUSDT_d.csv


In [5]:
df = pd.read_csv('Binance_BTCUSDT_d.csv', index_col=0, parse_dates=True)
df.dropna(axis=0, how='all', inplace=True)
df.dropna(axis=1, how='any', inplace=True)

In [6]:
df

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
Unix,,,,,,,,,
1674172800000,2023-01-20,BTCUSDT,21071.59,22755.93,20861.28,22667.21,338079.136590,7.256173e+09,7382435
1674086400000,2023-01-19,BTCUSDT,20677.47,21192.00,20659.19,21071.59,251385.849250,5.243024e+09,5819932
1674000000000,2023-01-18,BTCUSDT,21132.29,21650.00,20407.15,20677.47,350916.019490,7.399719e+09,7617173
1673913600000,2023-01-17,BTCUSDT,21185.65,21647.45,20841.31,21134.81,275407.744090,5.840586e+09,6440138
1673827200000,2023-01-16,BTCUSDT,20872.99,21474.05,20611.48,21185.65,293078.082620,6.168654e+09,6856434
...,...,...,...,...,...,...,...,...,...
1503273600000,2017-08-21,BTCUSDT,4069.13,4119.62,3911.79,4016.00,691.743060,2.797232e+06,3972
1503187200000,2017-08-20,BTCUSDT,4120.98,4211.08,4032.62,4086.29,467.083022,1.930364e+06,2321
1503100800000,2017-08-19,BTCUSDT,4108.37,4184.69,3850.00,4139.98,381.309763,1.549484e+06,2153


In [7]:
df['Volume USDT'].fillna(df['Volume USDT'].mean(), inplace=True)

In [8]:
df['Date'] = pd.to_datetime(df['Date'])

In [9]:
df['Date']

Unix
1674172800000   2023-01-20
1674086400000   2023-01-19
1674000000000   2023-01-18
1673913600000   2023-01-17
1673827200000   2023-01-16
                   ...    
1503273600000   2017-08-21
1503187200000   2017-08-20
1503100800000   2017-08-19
1503014400000   2017-08-18
1502928000000   2017-08-17
Name: Date, Length: 1983, dtype: datetime64[ns]

In [10]:
df = df.set_index('Date')


In [11]:
df = df.drop(columns=['Volume USDT'])


In [12]:
df

,Symbol,Open,High,Low,Close,Volume BTC,tradecount
Date,,,,,,,
2023-01-20,BTCUSDT,21071.59,22755.93,20861.28,22667.21,338079.136590,7382435
2023-01-19,BTCUSDT,20677.47,21192.00,20659.19,21071.59,251385.849250,5819932
2023-01-18,BTCUSDT,21132.29,21650.00,20407.15,20677.47,350916.019490,7617173
2023-01-17,BTCUSDT,21185.65,21647.45,20841.31,21134.81,275407.744090,6440138
2023-01-16,BTCUSDT,20872.99,21474.05,20611.48,21185.65,293078.082620,6856434
...,...,...,...,...,...,...,...
2017-08-21,BTCUSDT,4069.13,4119.62,3911.79,4016.00,691.743060,3972
2017-08-20,BTCUSDT,4120.98,4211.08,4032.62,4086.29,467.083022,2321
2017-08-19,BTCUSDT,4108.37,4184.69,3850.00,4139.98,381.309763,2153


In [13]:
#Calculate the returns
df['returns'] = df['Close'].pct_change()

In [14]:
df = df.apply(pd.to_numeric, errors='coerce')


In [15]:
print(np.isinf(df).sum())


Symbol        0
Open          0
High          0
Low           0
Close         0
Volume BTC    0
tradecount    0
returns       0
dtype: int64


In [16]:
#Apply the Savgol filter to the returns
window_size = 5
polyorder = 2
df['smoothed_returns'] = savgol_filter(df['returns'], window_size, polyorder)

In [17]:
df['smoothed_returns']

Date
2023-01-20         NaN
2023-01-19         NaN
2023-01-18         NaN
2023-01-17    0.012460
2023-01-16    0.004928
                ...   
2017-08-21   -0.000167
2017-08-20    0.013166
2017-08-19    0.006587
2017-08-18    0.016578
2017-08-17    0.031997
Name: smoothed_returns, Length: 1983, dtype: float64

In [18]:
# Define the replay buffer
replay_buffer = []

In [19]:
# Define the reward function
def reward_function(returns, smoothed_returns, gamma):
    rewards = []
    for i, r in enumerate(returns):
        future_returns = returns[i+1:]
        future_rewards = [np.power(gamma, j) * r_future for j, r_future in enumerate(future_returns)]
        total_future_reward = sum(future_rewards)
        rewards.append(total_future_reward)
    return rewards


In [20]:
# Define the discount factor
gamma = 0.9


In [21]:
# Calculate the rewards
df['rewards'] = reward_function(df['returns'], df['smoothed_returns'], gamma)

In [22]:
df['rewards']

Date
2023-01-20   -0.169854
2023-01-19   -0.110512
2023-01-18   -0.102009
2023-01-17   -0.137919
2023-01-16   -0.155916
                ...   
2017-08-21    0.054499
2017-08-20    0.041107
2017-08-19    0.031076
2017-08-18    0.043012
2017-08-17    0.000000
Name: rewards, Length: 1983, dtype: float64

In [23]:

# Add the rewards and returns to the replay buffer
replay_buffer.append((df['returns'], df['rewards']))

In [24]:
# Define the training and testing sets
train_start = datetime(2017,8,17)
train_end = datetime(2021,12,31)
test_start = datetime(2022,1,1)
test_end = datetime(2023,1,1)


In [25]:
# Create the training and testing sets
train_data = df[(df.index >= train_start) & (df.index <= train_end)]
test_data = df[(df.index >= test_start) & (df.index <= test_end)]


In [26]:
# Train the linear regression model using the replay buffer
X_train = train_data[['smoothed_returns']]
y_train = train_data['rewards']
model = LinearRegression().fit(X_train, y_train)


In [27]:
# Make predictions on the testing set
X_test = test_data[['smoothed_returns']]
y_test = test_data['rewards']



In [28]:
print(X_test.isnull().sum())


smoothed_returns    0
dtype: int64


In [29]:
print(np.isinf(X_test).sum())


smoothed_returns    0
dtype: int64


In [30]:
X_test = X_test.fillna(X_test.mean())


In [31]:
y_pred = model.predict(X_test)

In [32]:
# Evaluate the model's performance
mse = ((y_test - y_pred) ** 2).mean()
print("Mean Squared Error: ", mse)

Mean Squared Error:  0.006687333254723184


In [33]:

# Apply the model to new data to make predictions
current_returns = df.iloc[-1]['smoothed_returns']
future_reward = model.predict([[current_returns]])
print("Predicted future reward")

Predicted future reward


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [34]:
# Create a new dataframe for backtesting
df_backtest = df[df.index < test_start]


In [35]:
# Use the model to make predictions on the backtesting set
X_backtest = df_backtest[['smoothed_returns']]
y_backtest_pred = model.predict(X_backtest)


In [36]:
# Compare the predictions to the actual outcomes
df_backtest['predicted_rewards'] = y_backtest_pred
df_backtest['error'] = df_backtest['rewards'] - df_backtest['predicted_rewards']


<ipython-input-36-2c7d8636debf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_backtest['predicted_rewards'] = y_backtest_pred
<ipython-input-36-2c7d8636debf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_backtest['error'] = df_backtest['rewards'] - df_backtest['predicted_rewards']


In [37]:
# Calculate performance metrics
mse = ((df_backtest['error']) ** 2).mean()
print("Mean Squared Error: ", mse)

Mean Squared Error:  0.010615593616353195


In [38]:
# Calculate the return on investment
df_backtest['return_on_investment'] = df_backtest['returns'] * df_backtest['predicted_rewards']
roi = df_backtest['return_on_investment'].sum()
print("Return on Investment: ", roi)

Return on Investment:  1.0080377573309172


<ipython-input-38-1baf0912f30d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_backtest['return_on_investment'] = df_backtest['returns'] * df_backtest['predicted_rewards']
